# Indigenous lands on which observatories are located

In [1]:
import pandas as pd
import folium
import numpy as np
import math
from astropy.coordinates import EarthLocation as EL
import astropy.units as u
import requests

In [2]:
sites = EL.get_site_names()
sites_csv = pd.read_csv('Observatory Locations - Sheet1.csv')

## Get map

In [3]:
folium_map = folium.Map(location=[43.6532, -79.3832],
                        zoom_start=3,
                        tiles="https://tiles.stadiamaps.com/tiles/outdoors/{z}/{x}/{y}{r}.png",attr='http://openstreetmap.org')

In [17]:
territory_track = set() # hack for unique territory

#responses = requests.get("https://native-land.ca/wp-content/themes/Native-Land-Theme/files/indigenousTerritories.json")

for _, site in sites_csv.iterrows():
    site_name = site[0]
    loc =EL.from_geodetic(lat=site[1]*u.deg, lon=site[2]*u.deg).geodetic
    locstr = '{:.6f},{:.6f}'.format(loc[1].to(u.deg).value, loc[0].to(u.deg).value)

    response = requests.get("https://native-land.ca/api/index.php?maps=territories&position={:s}".format(locstr))
    infos = response.json()
    namelist = []
    for info in infos:
        territory_name = info['properties']['Name']
        territory_color = info['properties']['color']
        style_function = lambda x, fill_color = territory_color, color = territory_color: {'fillColor': fill_color, 'color': color}
        namelist.append(territory_name)
        if territory_name not in territory_track:
            folium.GeoJson(info['geometry'], name=territory_name, tooltip=namelist, style_function=style_function).add_to(folium_map)
            territory_track.add(territory_name)
    
    namestr = ''
    c = 0
    for name in namelist:
        if c==(len(namelist)-1):
            namestr=namestr+"{:s}".format(name)
        else:
            namestr=namestr+"{:s},\n".format(name)
        c+=1
    popup_text = "Observatory Name: {}<br> Indigenous Land: {}"
    popup_text = popup_text.format(site_name,
                          namestr)
    popup = folium.Popup(popup_text, max_width=200)

    folium.Marker(location = [loc[1].to(u.deg).value, loc[0].to(u.deg).value], popup=popup).add_to(folium_map)

In [18]:
folium_map

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

